In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
import plotly.express as px
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from yellowbrick.cluster import KElbowVisualizer
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_excel("rawdata.xlsx")

In [3]:
df.head(5)

,date,time,sensor,location,number,activity,position,location.1
0,2024-01-16,10:10:30,0.0,A1,1,placed,inside,A1
1,2024-01-16,10:12:30,1.0,A2,1,picked,inside,A2
2,2024-01-16,10:28:30,1.0,A3,1,picked,Inside,A3
3,2024-01-16,10:38:20,0.0,A4,1,placed,Inside,A4
4,2024-01-16,10:45:20,1.0,A5,1,placed,inside,A5


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 174
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      174 non-null    datetime64[ns]
 1   time      174 non-null    object        
 2   sensor    174 non-null    float64       
 3   location  174 non-null    object        
 4   number    174 non-null    int64         
 5   activity  174 non-null    object        
 6   position  174 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 10.9+ KB


In [4]:
df.nunique()

date            3
time          140
sensor          2
location       34
number          2
activity        2
position        3
location.1     34
dtype: int64

In [5]:
df=df.drop('location.1', axis=1)

In [6]:
df.dropna(inplace=True)

In [18]:
df['date'] = pd.to_datetime(df['date'])

In [20]:
inside_activities = df[df['position'].str.lower() == 'inside']
outside_activities = df[df['position'].str.lower() == 'outside']


In [26]:
inside_activities

,date,time,sensor,location,number,activity,position
0,2024-01-16,10:10:30,0.0,A1,1,placed,inside
1,2024-01-16,10:12:30,1.0,A2,1,picked,inside
2,2024-01-16,10:28:30,1.0,A3,1,picked,Inside
3,2024-01-16,10:38:20,0.0,A4,1,placed,Inside
4,2024-01-16,10:45:20,1.0,A5,1,placed,inside
...,...,...,...,...,...,...,...
169,2024-01-18,12:35:30,1.0,A7,2,picked,inside
170,2024-01-18,12:45:05,0.0,A8,2,placed,inside
171,2024-01-18,12:53:10,1.0,A9,2,picked,inside
173,2024-01-18,13:10:20,1.0,A11,2,picked,Inside


In [59]:
df['time'] = df['time'].astype(str)

In [60]:
df['time'] = pd.to_datetime(df['time'])

In [61]:
duration_df = df.groupby(['date', 'position'])['time'].apply(lambda x: (x.max() - x.min()).total_seconds() / 60).reset_index(name='total_duration')


In [62]:
duration_df

,date,position,total_duration
0,2024-01-16,Inside,9.833333
1,2024-01-16,inside,717.000000
2,2024-01-17,inside,190.833333
3,2024-01-18,Inside,385.500000
4,2024-01-18,inside,559.833333
5,2024-01-18,outside,357.833333


In [63]:
activity_counts = df.groupby(['date', 'activity']).size().reset_index(name='activity_count')


In [64]:
picking_placing_counts = activity_counts[activity_counts['activity'].isin(['picked', 'placed'])]


In [65]:
picking_placing_counts_pivot = picking_placing_counts.pivot(index='date', columns='activity', values='activity_count').fillna(0)


In [66]:
picking_placing_counts_pivot.reset_index(inplace=True)


In [67]:
picking_placing_counts_pivot.columns.name = None  # Remove column name
picking_placing_counts_pivot.columns = ['date', 'placing_count', 'picking_count']


In [68]:
print(picking_placing_counts_pivot)


        date  placing_count  picking_count
0 2024-01-16             40             40
1 2024-01-17             10              9
2 2024-01-18             37             38
